In [1]:
import pandas as pd
import numpy as np
import math
import time

In [3]:
Train_data = pd.read_csv(r'C:\Users\miriy\Downloads\heart_train.data',header = None)
Test_data = pd.read_csv(r'C:\Users\miriy\Downloads\heart_test.data',header = None)

In [4]:
Train_data.loc[Train_data[0] == 0, 0] = -1
Test_data.loc[Test_data[0] == 0, 0] = -1

In [5]:
attrs = len(Train_data.columns)  #No of Attributes

rows = len(Train_data)

In [6]:
Train_data = np.array(Train_data.iloc[:,:])
Test_data = np.array(Test_data.iloc[:,:])

In [7]:
def output(tree, lst):
    while(True):
        for key, value in tree.items():
            #print('key', key)
            #print('value', value)
            if value == 1 or value == -1:
                return value
            else:
                k = lst[key]
                tree = tree[key][k]
            if tree == 1 or tree == -1:
                return tree

In [8]:
def AdaBoost(rounds, model, data):
    weightedError = 1
    weight = [1/rows]*rows #Initial weight matrix
    alpha = []
    HSpace = [] #Total hypothesis space
    predictions = []
    labels = []
    eps = []    #Weighted errors
    for r in range(rounds):
        print('round',r+1)
        weightedError = 1
        it = 0
        for mod in model:
            w = 0
            if r == 0:
                labels.append([])
            lst = []
            for i in range(rows):
                row = data[i, :]
                if r == 0:
                    k = output(mod, row)
                    lst.append(k)
                    if row[0] != k:
                        w = w + weight[i]
                    labels[it].append(k)
                else:
                    k = labels[it][i]
                    lst.append(k)
                    if row[0] != k:
                        w = w + weight[i]
                    if w > weightedError or w > 0.5:#Optimization
                        break
            if w < weightedError:
                weightedError = w
                bTree = mod #bTree indicates it is a best tree
                bestP = lst
            it += 1
        HSpace.append(bTree)
        eps.append(weightedError)
        t = 1/2 * math.log((1-weightedError)/weightedError)
        alpha.append(t)
        #Weight Updation
        sum1 = 0
        predictions.append(bestP)
        for i in range(rows):
            prediction = bestP[i]
            actual = data[i, 0]
            #print(prediction, actual)
            weight[i] = (weight[i] * (np.exp(-1 * prediction * actual * t)))/(2*np.sqrt(weightedError * (1-weightedError)))
            sum1 += weight[i]
        print("sum", sum1)
        #print(weight)
        #print(alpha)
        print(weightedError)                                  
    return eps, alpha, HSpace, predictions

In [15]:
def Hypothesis():
    hypo = []
    lst = [1, -1]
    for i in range(1, attrs):#attribute iteration
        for j in range(1, attrs):
#             for k in range(1, attrs):#5 different cases for 3 attribute splits
#         for a in lst:
            for a in lst:
                for b in lst:
                    for c in lst:
                        tree = {}                #case 1 LL
                        tree[i] = {}
                        tree[i][1] = a
                        tree1 = {}
                        tree1[j] = {}
                        tree1[j][0] = b
                        tree1[j][1] = c
                        tree[i][0] = tree1
                        hypo.append(tree)
                        
                        tree = {}                #case 2 LR
                        tree[i] = {}
                        tree[i][0] = a
                        tree1 = {}
                        tree1[j] = {}
                        tree1[j][0] = b
                        tree1[j][1] = c
                        tree[i][1] = tree1
                        hypo.append(tree)
                         
    return hypo

In [16]:
rounds = 5
print('starting time')
t1 = time.time()
Thypo = Hypothesis()   #Thypo mean Total hypothesis space
print('end')
t2 = time.time()
print(t2-t1, 'secs')
#print(hypo)
print('Generated Hypothesis spaces')

epsilon, alpha, hypo, predictions = AdaBoost(rounds, Thypo, Train_data) #method for getting Hypothesis space and Alpha values
t2 = time.time()
print(t2-t1, 'secs')

print(hypo)

starting time
end
0.01997661590576172 secs
Generated Hypothesis spaces
round 1
sum 0.9999999999999993
0.21250000000000005
round 2
sum 1.0000000000000007
0.31932773109243706
round 3
sum 1.0000000000000009
0.3665619810843981
round 4
sum 1.0000000000000007
0.3073301040034191
round 5
sum 1.0000000000000009
0.3609222826701768
3.373041868209839 secs
[{11: {1: 1, 0: {13: {0: -1, 1: 1}}}}, {4: {1: 1, 0: {22: {0: -1, 1: 1}}}}, {3: {0: 1, 1: {20: {0: -1, 1: 1}}}}, {8: {1: 1, 0: {16: {0: -1, 1: 1}}}}, {3: {0: 1, 1: {20: {0: -1, 1: 1}}}}]


In [18]:
length = len(Test_data)

accuracy = 0
for i in range(length):
    p = 0
    row = Test_data[i, :]
    y = row[0]
    for r in range(rounds):
        k = output(hypo[r], row)
        p = p + alpha[r] * k
    if p >= 0:
        if y == 1:
            accuracy += 1
    else:
        if y == -1:
            accuracy += 1

accuracy = accuracy/length * 100
print("Accuracy on the test data set is", accuracy)

Accuracy on the test data set is 76.47058823529412


In [19]:
for r in range(rounds):
    print('The value of epsilon and alpha for round,',r+1,'is',alpha[r],epsilon[r])
    
t2 = time.time()
print(t2-t1, 'secs')

The value of epsilon and alpha for round, 1 is 0.6549606911676581 0.21250000000000005
The value of epsilon and alpha for round, 2 is 0.37843149747302635 0.31932773109243706
The value of epsilon and alpha for round, 3 is 0.2734972663157064 0.3665619810843981
The value of epsilon and alpha for round, 4 is 0.4063155589491071 0.3073301040034191
The value of epsilon and alpha for round, 5 is 0.2856817093262064 0.3609222826701768
76.26973509788513 secs


In [17]:
hypo

[{11: {1: 1, 0: {13: {0: -1, 1: 1}}}},
 {4: {1: 1, 0: {22: {0: -1, 1: 1}}}},
 {3: {0: 1, 1: {20: {0: -1, 1: 1}}}},
 {8: {1: 1, 0: {16: {0: -1, 1: 1}}}},
 {3: {0: 1, 1: {20: {0: -1, 1: 1}}}}]